# Advanced Lane Lines Finding Project

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib inline

## Camera calibration matrix and distortion correction

In [ ]:
def calibration():
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = mpimg.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            calibrated_img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
            mpimg.imsave('output_images/calibration/' + fname[10:], calibrated_img)
            
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist


def distortion_correction(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

## Thresholded binary image

In [ ]:
def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        sobel_parameter = [1, 0]
    elif orient == 'y':
        sobel_parameter = [0, 1]
    else:
        return "Wrong Orient"
        
    sobel = cv2.Sobel(gray, cv2.CV_64F, sobel_parameter[0], sobel_parameter[1], ksize=sobel_kernel)
    
    abs_sobel = np.absolute(sobel)
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    return grad_binary


def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    mag = np.sqrt(sobelx**2 + sobely**2)

    scaled_mag = np.uint8(255*mag/np.max(mag))

    mag_binary = np.zeros_like(scaled_mag)
    mag_binary[(scaled_mag >= mag_thresh[0]) & (scaled_mag <= mag_thresh[1])] = 1
    
    return mag_binary


def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    
    dir_grad = np.arctan2(abs_sobely, abs_sobelx)
    
    dir_binary = np.zeros_like(dir_grad)
    dir_binary[(dir_grad >= thresh[0]) & (dir_grad <= thresh[1])] = 1
    
    return dir_binary


def color_select(image, r_thresh=(0, 255), g_thresh=(0, 255), l_thresh=(0, 255), s_thresh=(0, 255)):
    r_channel = image[:,:,0]
    g_channel = image[:,:,1]
    
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel > r_thresh[0]) & (r_channel <= r_thresh[1])] = 1
    
    g_binary = np.zeros_like(g_channel)
    g_binary[(g_channel > g_thresh[0]) & (g_channel <= g_thresh[1])] = 1
    
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel > l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    return r_binary, g_binary, l_binary, s_binary


def combined_color_threshold(image):
    blur_img = cv2.GaussianBlur(image, (3, 3), 0)
    r_binary, g_binary, l_binary, s_binary = \
    color_select(blur_img, r_thresh=(150, 255), g_thresh=(150, 255), l_thresh=(120, 255), s_thresh=(100, 255))
    
    # Sobel kernal size
    ksize = 3 
    
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(blur_img, orient='x', sobel_kernel=ksize, thresh=(40, 200))

    dir_binary = dir_threshold(blur_img, sobel_kernel=ksize, thresh=(np.pi/6, np.pi/2))

    combined_thresh = np.zeros_like(dir_binary)
    combined_thresh[((gradx == 1) & (dir_binary == 1))] = 1
    
    combined_color = np.zeros_like(r_binary)
    combined_color[(r_binary == 1) & (g_binary == 1) & (l_binary == 1)] = 1
    
    combined_binary = np.zeros_like(combined_color)
    combined_binary[(combined_color == 1) & ((combined_thresh == 1) | (s_binary == 1))] = 1
    
    # Apply a triangle mask
    mask = np.zeros_like(combined_binary)
    h = image.shape[0]
    w = image.shape[1]
    mask_resion = [np.array([[0, h-1], [w//2, h//2],[w-1, h-1]], dtype=np.int32)]
    cv2.fillPoly(mask, mask_resion, 1)
    
    combined_mask = cv2.bitwise_and(combined_binary, mask)
    
    return combined_mask

## birds-eye view

In [ ]:
def birds_eye(image):
#     offset = 0
    x = image.shape[1]
    y = image.shape[0]
    
#     src = np.float32([
#         [530, 497],
#         [0, 670],
#         [1280, 670],
#         [720, 450]
#     ])
    
#     dst = np.float32([
#         [offset, offset],
#         [offset, y-offset], 
#         [x-offset, y-offset], 
#         [x-offset, offset]
#     ])


    src = np.float32(
        [[(x / 2) - 63, y / 2 + 100],
        [((x / 6) - 20), y],
        [(x * 5 / 6) + 60, y],
        [(x / 2 + 65), y / 2 + 100]])
    dst = np.float32(
        [[(x / 4), 0],
        [(x / 4), y],
        [(x * 3 / 4), y],
        [(x * 3 / 4), 0]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(image, M, [x, y])
    
    return warped, M, M_inv

## Lane boundary

### First frame

In [ ]:
def hist(image):
    bottom_half = image[image.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram


def find_lane_pixels(image):
    # number of sliding windows
    windows = 9
    # the width of the windows +/- margin
    margin = 100
    # minimum number of pixels found to recenter window
    minpix = 5
    
    left_lane_inds = []
    right_lane_inds = []
    
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    out_img = np.dstack((image, image, image))
    
    histogram = hist(image)
    
    midpoint = int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Current positions to be updated later for each window in windows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Set height of windows
    window_height = int(image.shape[0]//windows)
    #x and y positions of all nonzero pixels in the image
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    for window in range(windows):
        # window boundaries in x and y (and right and left)
        win_y_low = image.shape[0] - (window+1)*window_height
        win_y_high = image.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If pixels > minpix, recentering next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = int(np.mean(nonzerox[good_right_inds]))
    
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(image):
    # lane pixels
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(image)
    
    # fitting second order polynomial
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generating x and y values for plotting
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    # Visualization
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    for i in range(image.shape[0]):
        cv2.circle(out_img, (int(left_fitx[i]), int(ploty[i])), 1, (255,255,0))
        cv2.circle(out_img, (int(right_fitx[i]), int(ploty[i])), 1, (255,255,0))

    return out_img, left_fit, right_fit

### Search in a margin around the previous lane line position

In [ ]:
def fit_poly(image, leftx, lefty, rightx, righty):
    img_shape = image.shape
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])

    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty


def line_average(prev, new):
    # Number of frames to consider
    frames = 12
    
    if new is None:
        if len(prev) == 0:
            return prev, None
        else:
            return prev, prev[-1]
        
    else:
        if len(prev) < frames:
            prev.append(new)
            return prev, new
        else:
            prev[0:frames-1] = prev[1:]
            prev[frames-1] = new
            new = np.zeros_like(new)
            for i in range(frames):
                new += prev[i]
                
            new /= frames
            return prev, new


def search_around_poly(image, left_fit=None, right_fit=None, prev_left=[], prev_right=[], mean=0):
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100
    
    # In case there is no fitted line
    if left_fit is None or right_fit is None:
        return fit_polynomial(image)

    # Grab activated pixels
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # In case there is no fitted line
    if leftx.size == 0 or rightx.size == 0:
        return fit_polynomial(image)

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(image, leftx, lefty, rightx, righty)
    
    # Finding the mean
    current_mean = np.mean(right_fitx - left_fitx)
    
    if mean == 0:
        mean = current_mean
        
    if current_mean < 0.7*mean or current_mean > 1.3*mean:
        if len(prev_left) == 0 and len(prev_right) == 0:
            return fit_polynomial(image)
        else:
            left_fitx = prev_left[-1]
            right_fitx = prev_right[-1]
              
    else:
        prev_left, left_fitx = line_average(prev_left, left_fitx)
        prev_right, right_fitx = line_average(prev_right, right_fitx)
        
        current_mean = np.mean(right_fitx - left_fitx)
        mean = 0.9*mean + 0.1*current_mean
    
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((image, image, image))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    for i in range(image.shape[0]):
        cv2.circle(result, (int(left_fitx[i]), int(ploty[i])), 1, (255,255,0))
        cv2.circle(result, (int(right_fitx[i]), int(ploty[i])), 1, (255,255,0))
    
    return result, left_fitx, right_fitx

## Measuring curvature and car position

In [ ]:
def measure_curvature(image, leftx, rightx):
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    img_shape = image.shape
    
    ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])
    
    xm_per_pix=3.7/700
    ym_per_pix = 15.0/img_shape[0]
    
    # Reverse to match top-to-bottom in y
    leftx = leftx[::-1]
    rightx = rightx[::-1]  
    
    # second order polynomial to pixel positions
    left_fit = np.polyfit(ploty, leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(ploty, rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # new polynomials to x,y in world space
    y_eval = np.max(ploty)
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad


def car_position(image, leftx, rightx, xm_per_pix=3.7/700):
#     gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    img_shape = image.shape
        
    ## Car position
    car_pos = (leftx[-1] + rightx[-1])/2 
    position = (img_shape[1]//2 - car_pos) * xm_per_pix

    return position

## Inverse Transform

In [ ]:
def draw_on_image(image, warped_img, left_fitx, right_fitx, M_inv, left_curverad, right_curverad, position):
#     gray = cv2.cvtColor(warped_img, cv2.COLOR_RGB2GRAY)
    
    ploty = np.linspace(0, image.shape[0]-1, image.shape[0])
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, (image.shape[1], image.shape[0]))
    
    # Display information on image
    left_curvature = "Left  Curvature: {:.2f} m".format(left_curverad) 
    right_curvature = "Right  Curvature: {:.2f} m".format(right_curverad)
    car_position = "Car offset: {:.2f} m".format(position)
    cv2.putText(newwarp, left_curvature, (100,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255),
                5,  lineType = cv2.LINE_AA)
    cv2.putText(newwarp, right_curvature, (100,120), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255),
                5,  lineType = cv2.LINE_AA)
    cv2.putText(newwarp, car_position, (100,190), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255),
                5,  lineType = cv2.LINE_AA)


    # Combine the result with the original image
    return cv2.addWeighted(image, 1, newwarp, 0.3, 0)

## Pipeline

In [ ]:
class Line():
    def __init__(self):
        # Initialization
        self.left_fit = None
        self.left_fitx = None
        self.right_fit = None
        self.right_fitx = None
        self.prev_left = []
        self.prev_right = []
        self.mean = 0
        
        # Camera calibration
        self.mtx, self.dist = calibration()
        
    def __call__(self, image):
        # Distortion correction
        undist_img = distortion_correction(image, self.mtx, self.dist)
        
        # Threshold binary
        combined_mask_img = combined_color_threshold(undist_img)
        
        # Prespective transform
        birds_eye_img, M, M_inv = birds_eye(combined_mask_img)
        
        # Lane boundry
        fit_img, self.left_fit, self.right_fit = fit_polynomial(birds_eye_img)
        search_img, self.left_fitx, self.right_fitx = search_around_poly(birds_eye_img, self.left_fit, self.right_fit)
        
        # Curvature and car position
        left_curverad, right_curverad = measure_curvature(birds_eye_img, self.left_fitx, self.right_fitx)
        position = car_position(birds_eye_img, self.left_fitx, self.right_fitx)
        
        # Inverse transform
        result = draw_on_image(undist_img, birds_eye_img, self.left_fitx, self.right_fitx,
                      M_inv, left_curverad, right_curverad, position)
        
        return result

In [ ]:
line = Line()

imgs = os.listdir('test_images')

for img in imgs:
    test_img = mpimg.imread('test_images/' + img)
    result = line(test_img)
    
    mpimg.imsave('output_images/pipeline/' + img, result)

In [ ]:
from moviepy.editor import VideoFileClip

video_frame = Line()

clip1 = VideoFileClip('./project_video.mp4')
white_clip = clip1.fl_image(video_frame)
%time white_clip.write_videofile('./output_videos/project_video_output.mp4', audio=False)